In [1]:
!pip install bs4

In [2]:
!pip install requests

In [3]:
!pip install pandas

In [5]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import re

In [6]:
#function to extract product label
def get_title(soup):
    
    try:
        
        title = soup.find("title", attrs = {}).text.strip()
        
    except AttributeError:
        title = ""
    
    return title

In [7]:
#function to extract Product Offer Price

def get_offerprice(soup):
    
    try:
        
        price_text = soup.find("span", attrs = {"class": "offer"}).text.strip()
        
        offer_price = re.sub(r'^.*?(\d+\.\d+ CAD)', r'\1', price_text)
        
    except AttributeError:
        offer_price = ""
        
    return offer_price

In [17]:
#function to extract the rating of a product

def get_rating(soup):
    
    try:
        
        review_rating = soup.find("div", attrs = {"class": "sc-jtRfpW burazA"}).text.strip()
        
        rating = re.sub(r'[^0-9.]', '', review_rating)
    
    except AttributeError:
        
        rating = ""
    
    return rating
        

In [ ]:
# #function to extract reviews

# def get_review(soup):
    
#     try:
#         review = soup.find("div", attrs = {"class": "reviews_count"}).find("span", attrs ={})
        
#     except AttributeError:
#         review = ""
    
#     return review

In [ ]:
if __name__ == '__main__':
    
    #URL
    URL = 'https://www.eddiebauer.ca/c/20001/men?cm_sp=topnav_m_featured_viewall'
    
    #user agent
    HEADERS = ({'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36','Accept-Language': 'en-US, en;q=0.5'})
    
    #HTTP Request
    session = requests.Session()
    webpage = session.get(URL, headers = HEADERS)
    
    #Soup Object
    soup = BeautifulSoup(webpage.content, 'html.parser')
    
    #Fetch links
    links = soup.find_all("a", attrs = {'class': 'wrapper'})
    
    #Store all links in a list
    links_list = []
    
    #extract links to the list
    for link in links:
        links_list.append(link.get('href'))
    
    dic = {"title":[], "offer_price":[], "ratings": []}
    
    #Extracting product details
    for link in links_list:
        product_link = "https://www.eddiebauer.ca" + link
        new_webpage = session.get(product_link, headers = HEADERS)
        
        new_soup = BeautifulSoup(new_webpage.content, "html.parser")
        
        #function calls to display product information
        dic['title'].append(get_title(new_soup))
        dic['offer_price'].append(get_offerprice(new_soup))
        dic['ratings'].append(get_rating(new_soup))
#         dic['reviews'].append(get_review(new_soup))
        
        
        data_df = pd.DataFrame.from_dict(dic)
        data_df['title'].replace('', np.nan, inplace=True)
        data_df = data_df.dropna(subset = ["title"])
        data_df.to_csv("Eddie_Bauer_data.csv", header=True, index=False)

In [23]:
data_df


,title,offer_price,ratings
0,Men's Powder Search 2.0 3-in-1 Jacket | Eddie ...,275.40 CAD,4.75
1,Men's Powder Search 2.0 3-in-1 Jacket | Eddie ...,275.40 CAD,4.75
2,Men's Eddie's Favorite Ultrasoft Thermal Henle...,42.00 CAD,4.75
3,Men's Eddie's Favorite Ultrasoft Thermal Henle...,42.00 CAD,4.75
4,Men's Superior Down Parka | Eddie Bauer,269.40 CAD,4.65
...,...,...,...
139,Men's Faultline Corduroy Shirt | Eddie Bauer,72.00 CAD,4.75
140,Men's Ignitelite Stretch Reversible Hooded Jac...,179.40 CAD,4.75
141,Men's Ignitelite Stretch Reversible Hooded Jac...,179.40 CAD,4.75
142,Men's Astrolite Hooded Jacket | Eddie Bauer,155.40 CAD,4.55


In [28]:
data_df.to_csv("Scraped_Data.csv", index = "false")